In [ ]:
!pip install opencv-python
!pip install numpy


In [ ]:
import os

# Create a directory to store YOLO model files
os.makedirs("yolo-coco", exist_ok=True)

# Download YOLO model weights
!wget -q -O yolo-coco/yolov3.weights https://pjreddie.com/media/files/yolov3.weights

# Download YOLO model configuration
!wget -q -O yolo-coco/yolov3.cfg https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

# Download COCO names file
!wget -q -O yolo-coco/coco.names https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


# Will print some sample from each of the directories in yolo-coco

In [ ]:
import os

# List the contents of the yolo-coco directory
print("Contents of yolo-coco directory:")
print(os.listdir("yolo-coco"))

# Print the size of the weights file
weights_path = "yolo-coco/yolov3.weights"
weights_size = os.path.getsize(weights_path)
print(f"\nSize of yolov3.weights: {weights_size} bytes")

# Print the first 10 lines of the configuration file
cfg_path = "yolo-coco/yolov3.cfg"
print("\nFirst 10 lines of yolov3.cfg:")
with open(cfg_path, 'r') as f:
    for i in range(10):
        print(f.readline().strip())

# Print all class labels from coco.names
names_path = "yolo-coco/coco.names"
print("\nClass labels in coco.names:")
with open(names_path, 'r') as f:
    class_labels = f.read().strip().split('\n')
    for label in class_labels:
        print(label)


In [ ]:
import cv2
import numpy as np

class Config:
    MODEL_PATH = "yolo-coco"
    MIN_CONF = 0.3
    NMS_THRESH = 0.3
    USE_GPU = True
    MIN_DISTANCE = 50

# Load YOLO model
# cv2.dnn.readNetFromDarknet: Loads the YOLO model from the specified configuration and weights files.
# os.path.join(Config.MODEL_PATH, "yolov3.cfg"): Constructs the path to the YOLO configuration file.
# os.path.join(Config.MODEL_PATH, "yolov3.weights"): Constructs the path to the YOLO weights file.
# net: The loaded YOLO mode
net = cv2.dnn.readNetFromDarknet(
    os.path.join(Config.MODEL_PATH, "yolov3.cfg"),
    os.path.join(Config.MODEL_PATH, "yolov3.weights")
)
# labelsPath: Constructs the path to the COCO class labels file.
# open(labelsPath).read(): Reads the contents of the COCO names file.
# strip().split("\n"): Splits the contents by newline characters to create a list of class labels.
# LABELS: A list containing the names of the classes that YOLO can detect, such as "person", "bicycle", "car", etc
labelsPath = os.path.join(Config.MODEL_PATH, "coco.names")
LABELS = open(labelsPath).read().strip().split("\n")

# Get YOLO output layer names
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# Set the preferred backend and target to GPU if configured
if Config.USE_GPU:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


In [ ]:
def detect_people(frame, net, ln, personIdx=0):
    (H, W) = frame.shape[:2]
    results = []

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    boxes = []
    centroids = []
    confidences = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if classID == personIdx and confidence > Config.MIN_CONF:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, Config.MIN_CONF, Config.NMS_THRESH)

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    return results

def euclidean_dist(pt1, pt2):
    return np.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)


In [ ]:
# Open the video file
video_path = "/content/pedestrians.mp4"
vs = cv2.VideoCapture(video_path)

output_path = "output_video.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = None
(W, H) = (None, None)

while True:
    (grabbed, frame) = vs.read()
    if not grabbed:
        break

    results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

    violate = set()

    if len(results) >= 2:
        centroids = np.array([r[2] for r in results])
        D = np.zeros((len(centroids), len(centroids)), dtype="float")

        for i in range(0, len(centroids)):
            for j in range(i + 1, len(centroids)):
                D[i, j] = euclidean_dist(centroids[i], centroids[j])

                if D[i, j] < Config.MIN_DISTANCE:
                    violate.add(i)
                    violate.add(j)

    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid

        color = (0, 255, 0)
        if i in violate:
            color = (0, 0, 255)

        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.circle(frame, (cX, cY), 5, color, 1)

    text = "Social Distancing Violations: {}".format(len(violate))
    cv2.putText(frame, text, (10, frame.shape[0] - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

    if writer is None:
        (H, W) = frame.shape[:2]
        writer = cv2.VideoWriter(output_path, fourcc, 25, (W, H), True)

    writer.write(frame)

writer.release()
vs.release()


In [ ]:
from google.colab import files
files.download(output_path)
